## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 🔗 Shared State Middleware

## Industry Use Case: Transaction Audit Trail

This notebook demonstrates how middleware can **share state** and communicate.

| Feature | FSI Application |
|---------|-----------------|
| **State Sharing** | Track transaction counts |
| **Cross-Middleware Communication** | Pass audit data |
| **Metrics Collection** | Usage statistics |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")

In [ ]:
from collections.abc import Awaitable, Callable

from agent_framework import FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define Transaction Tools

In [ ]:
def check_balance(account_id: str) -> str:
    """Check account balance."""
    balances = {"ACC-001": 15432.50, "ACC-002": 8750.25, "ACC-003": 42100.00}
    balance = balances.get(account_id, 0.00)
    return f"Account {account_id}: Balance is ${balance:,.2f}"


def transfer_funds(from_account: str, to_account: str, amount: float) -> str:
    """Transfer funds between accounts."""
    return f"Transferred ${amount:,.2f} from {from_account} to {to_account}"


print("✅ Tools defined: check_balance, transfer_funds")

## Middleware Container with Shared State

Multiple middleware functions share state via a container class:
- **call_counter_middleware**: Tracks total function calls
- **audit_enhancer_middleware**: Adds audit metadata to results

In [ ]:
class AuditTrailMiddleware:
    """Container class holding middleware functions with shared audit state."""

    def __init__(self) -> None:
        self.call_count: int = 0
        self.audit_log: list[str] = []

    async def call_counter_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """First middleware: tracks function calls."""
        self.call_count += 1
        function_name = context.function.name if context.function else "unknown"
        print(f"[AuditCounter] Call #{self.call_count}: {function_name}")
        
        # Log to audit trail
        self.audit_log.append(f"Call #{self.call_count}: {function_name}")
        
        await next(context)

    async def audit_enhancer_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """Second middleware: adds audit metadata to results."""
        await next(context)
        
        # Enhance result with audit info
        if context.result:
            enhanced = f"[Audit #{self.call_count}] {context.result}"
            context.result = enhanced
            print(f"[AuditEnhancer] Added audit tag #{self.call_count}")


print("✅ AuditTrailMiddleware defined")

## Example: Transaction Audit Trail

In [ ]:
async def run_audit_example():
    """Shared state middleware for transaction audit trail."""
    print("\n--- Transaction Audit Trail Example ---\n")

    # Create middleware container with shared state
    audit = AuditTrailMiddleware()
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="BankingAgent",
            instructions="You help with banking transactions. Use tools to check balances and transfer funds.",
            tools=[check_balance, transfer_funds],
            middleware=[
                audit.call_counter_middleware,
                audit.audit_enhancer_middleware,
            ],
        ) as agent,
    ):
        queries = [
            "What's the balance of ACC-001?",
            "Transfer $500 from ACC-001 to ACC-002",
            "Check balance of ACC-002",
        ]

        for i, query in enumerate(queries, 1):
            print(f"\n--- Query {i} ---")
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}")

        # Display audit summary
        print("\n=== Audit Summary ===")
        print(f"Total function calls: {audit.call_count}")
        print("Audit log:")
        for entry in audit.audit_log:
            print(f"  - {entry}")

await run_audit_example()

## Key Takeaways

| Technique | When to Use | FSI Example |
|-----------|-------------|-------------|
| Container class | Group related middleware | Audit trail components |
| Instance variables | Share state | `call_count`, `audit_log` |
| Method-based middleware | Access `self` | Read/write shared state |
| Middleware order | Sequential processing | Counter → Enhancer |

## Summary: All Middleware Patterns

| # | Pattern | Purpose |
|---|---------|---------|
| 1 | Agent/Run Level | Apply globally or per-run |
| 2 | Function-based | Simple async functions |
| 3 | Class-based | `AgentMiddleware`, `FunctionMiddleware` |
| 4 | Decorator | `@agent_middleware`, `@function_middleware` |
| 5 | Chat | Message-level interception |
| 6 | Exception Handling | Catch and handle errors |
| 7 | Termination | Early exit with `context.terminate` |
| 8 | Result Override | Modify responses |
| 9 | Shared State | Cross-middleware communication |